In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from os import listdir
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [0]:
light_path = '/content/drive/My Drive/CV982/Sabok/'
heavy_path = '/content/drive/My Drive/CV982/Sangin/'
light_img, heavy_img = [], []

In [0]:
for i, each in enumerate(listdir(heavy_path)):
    print(i, 'H')
    try:
        img= Image.open(heavy_path + each)
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            heavy_img.append(img)
    except:
        print("ignored loading", each)
print(len(heavy_img), 'heavy done')
for i, each in enumerate(listdir(heavy_path + 'Bus/')):
    print(i, 'H')
    try:
        img= Image.open(heavy_path + 'Bus/' + each)
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            heavy_img.append(img)
    except:
        print("ignored loading", each)
print(len(heavy_img), 'heavy done')

In [0]:
for i, each in enumerate(listdir(light_path)):
    print(i, 'L')
    try:
        img= Image.open(light_path + each)
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            light_img.append(img)
    except:
        print("ignored loading", each)
print(len(light_img), 'light done')

In [0]:
# light_folders = ['BVMMR v11', 'BVMMR v12', 'BVMMR v13', 'BVMMR v14', 'BVMMR v15', 'BVMMR v16', 'BVMMR v17', 'BVMMR v18', 'BVMMR v19']
light_folders = ['BVMMR v11']
inner_folders =  ['Daewoo Cielo', 'Mazda Truck', 'Peugeot 206', 'Peugeot 207', 'Peugeot 405', 'Peugeot Pars', 'Peykan 60', 'Peykan 80', 'Peykan Truck', 'Pride 111', 'Pride 131', 'Pride 132', 'Pride 141', 'Renault L90', 'Rio', 'Samand', 'Tiba', 'Xantia', 'Zamyad Truck']
for i, each_folder in enumerate(light_folders):
    for each_inner in inner_folders:
        img_address = '/content/drive/My Drive/CV982/Sabok/' + each_folder + '/BVMMR v1/' + each_inner + '/'
        for each_img in listdir(img_address):
            if each_img[-3:] != 'ini':
                try:
                    img= Image.open(img_address + each_img).convert('RGB')
                    img = img.resize((224, 224))
                    img= np.array(img) / 255.
                    if img.shape == (224, 224, 3):
                        light_img.append(img)
                except:
                    print("ignored loading", img_address)
    print('Folder', i + 1, 'done.')
print(len(light_img), 'Iranian light done')

In [0]:
X = light_img + heavy_img
y = [0] * len(light_img) + [1] * len(heavy_img) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True)
print(len(X_train), len(y_train), len(X_val), len(y_val), len(X_val), len(y_val))

940 940 105 105 105 105


In [0]:
X_train = np.array(X_train, dtype=np.uint16)
y_train = np.array(y_train, dtype=np.uint16)

In [0]:
X_val = np.array(X_val, dtype=np.uint16)
y_val = np.array(y_val, dtype=np.uint16)

In [0]:
X_test = np.array(X_test, dtype=np.uint16)
y_test = np.array(y_test, dtype=np.uint16)

In [0]:
X_train = X_train.reshape(-1, 224, 224, 3)
X_val = X_val.reshape(-1, 224, 224, 3)
X_test = X_test.reshape(-1, 224, 224, 3)
print('Test :', X_train.shape, y_train.shape)
print('Val :', X_val.shape, y_val.shape)
print('Test :', X_test.shape, y_test.shape)

Test : (940, 224, 224, 3) (940,)
Val : (105, 224, 224, 3) (105,)
Test : (117, 224, 224, 3) (117,)


In [0]:
model = VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for each_layer in model.layers:
    each_layer.trainable = False
interaction_output = model.layers[-1].output
x = tf.keras.layers.Flatten()(interaction_output)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(model.input, x)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/CV982/model.h5", monitor='val_acc', save_best_only=True, mode='max')
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_data = (X_val, y_val), batch_size = 64, callbacks=[checkpoint])
model.save('/content/drive/My Drive/CV982/mainModel.h5' )

Train on 940 samples, validate on 105 samples
Epoch 1/100
940/940 [==============================] - 7s 8ms/sample - loss: 3.9730 - acc: 0.5372 - val_loss: 0.6090 - val_acc: 0.7048
Epoch 2/100
940/940 [==============================] - 4s 4ms/sample - loss: 0.5830 - acc: 0.6851 - val_loss: 0.5593 - val_acc: 0.7143
Epoch 3/100
940/940 [==============================] - 4s 4ms/sample - loss: 0.4698 - acc: 0.7468 - val_loss: 0.4768 - val_acc: 0.7429
Epoch 4/100
940/940 [==============================] - 3s 3ms/sample - loss: 0.4161 - acc: 0.7957 - val_loss: 0.4756 - val_acc: 0.7333
Epoch 5/100
940/940 [==============================] - 4s 4ms/sample - loss: 0.3819 - acc: 0.8309 - val_loss: 0.4611 - val_acc: 0.7524
Epoch 6/100
940/940 [==============================] - 3s 3ms/sample - loss: 0.3660 - acc: 0.8394 - val_loss: 0.4627 - val_acc: 0.7524
Epoch 7/100
940/940 [==============================] - 4s 4ms/sample - loss: 0.3399 - acc: 0.8649 - val_loss: 0.4643 - val_acc: 0.7619
Epoch 8/1

In [0]:
model.save('/content/drive/My Drive/CV982/mainModel.h5' )

In [0]:
_, acc = model.evaluate(X_train, y_train)
print('Train Accuracy: %.2f' % (acc*100))
_, acc = model.evaluate(X_test, y_test)
print('Test Accuracy: %.2f' % (acc*100))

940/940 [==============================] - 3s 3ms/sample - loss: 0.0723 - acc: 0.9521
Train Accuracy: 95.21
117/117 [==============================] - 1s 7ms/sample - loss: 0.8058 - acc: 0.7692
Test Accuracy: 76.92


In [0]:
model = load_model('/content/drive/My Drive/CV982/mainModel.h5')

In [0]:
def heavy_light_classifier(address):
    answer = None
    try:
        img= Image.open(address).convert('RGB')
        img = img.resize((224, 224))
        img= np.array(img) / 255.
        if img.shape == (224,224,3):
            prediction = model.predict(np.array([img]))[0][0]
            answer = round(prediction)
            answer = 'سبک' if answer == 0 else 'سنگین'
            answer += ' ' + str(prediction)
    except:
        print("ignored loading", each)
    return answer

In [20]:
test_img_address = 'zantia.jpg'
print(heavy_light_classifier(test_img_address))

سبک 0.45406777
